In [ ]:
# Connecting Azure Databricks to Azure Blob Storage

storage_account_name = ""
storage_account_key = ""
container = ""

spark.conf.set(f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net", storage_account_key)


In [ ]:
dbutils.fs.ls(f"wasbs://{container}@databricksdemostorage.blob.core.windows.net/")

In [ ]:
%fs ls wasbs://input@databricksdemostorage.blob.core.windows.net/

In [ ]:
df = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/retrain_housing_data.csv")

df.show()

In [ ]:
dbutils.fs.mount(
  source = f"wasbs://{container}@{storage_account_name}.blob.core.windows.net",
  mount_point = "/mnt/azureStorage",
  extra_configs = {f"fs.azure.account.key.{storage_account_name}.blob.core.windows.net": storage_account_key}
)

In [ ]:
dbutils.fs.mounts()

In [ ]:
#dbutils.fs.unmount("/mnt/azureStorage")

In [ ]:
%fs ls "dbfs:/mnt/azureStorage"

In [ ]:
df2 = spark.read \
  .option("header", "true") \
  .option("inferSchema", "true") \
  .option("delimiter", ",") \
  .csv("/mnt/azureStorage/retrain_housing_data.csv")

df2.show()

In [ ]:
data = spark.createDataFrame([(-114.42, 35.1, 11.0, 325.0, 94.0, 1212.0, 52.0, 5.22, 301500, "INLAND")], ["longitude", "latitude", "housing_median_age", "total_rooms", "total_bedrooms", "population", "households", "median_income", "median_house_value", "ocean_proximity"])

In [ ]:
data.show()

In [ ]:
data.write.csv(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/new")

In [ ]:
dbutils.fs.ls(f"wasbs://{container}@{storage_account_name}.blob.core.windows.net/")

In [ ]:
pandas_df = data.toPandas()
pandas_df.to_csv("/dbfs/FileStore/tables/pandas.csv", encoding="UTF-8")

In [ ]:
pandas_df.head()

In [ ]:
%fs ls dbfs:/FileStore/tables

In [ ]:
dbutils.library.installPyPI("azure.storage.blob")

In [ ]:
from azure.storage.blob import BlobClient

In [ ]:
connection = ""

In [ ]:
client = BlobClient.from_connection_string(connection, container_name="", blob_name="pandas.csv")

In [ ]:
with open("/dbfs/FileStore/tables/pandas.csv", "rb") as f:
  client.upload_blob(f)

In [ ]:
%fs ls wasbs://input@databricksdemostorage.blob.core.windows.net/

In [ ]:
new_data = client.download_blob()

In [ ]:
new_data.content_as_text()